# Data Cleaning

Preparing the data for modeling by handling the multi-row-per-inspection structure.

In [1]:
import pandas as pd
import numpy as np

# Load
df = pd.read_csv('../data/nyc_restaurant_inspections_raw.csv')
print(f"Original shape: {df.shape}")

Original shape: (295831, 27)


C:\Users\hoang\AppData\Local\Temp\ipykernel_37468\2487206695.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/nyc_restaurant_inspections_raw.csv')


In [2]:
# Convert date
df['INSPECTION DATE'] = pd.to_datetime(df['INSPECTION DATE'], errors='coerce')

# Remove uninspected
df_clean = df[df['INSPECTION DATE'].dt.year > 1900].copy()
print(f"After removing uninspected: {df_clean.shape}")

After removing uninspected: (292567, 27)


In [3]:
# Keep only graded inspections
df_clean = df_clean[df_clean['GRADE'].isin(['A', 'B', 'C'])].copy()
print(f"After filtering to A/B/C grades: {df_clean.shape}")

After filtering to A/B/C grades: (130520, 27)


In [4]:
# Check how many violations per inspection
violations_per_inspection = df_clean.groupby(['CAMIS', 'INSPECTION DATE']).size()
print(violations_per_inspection.describe())

count    51854.000000
mean         2.517067
std          1.334037
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max         15.000000
dtype: float64


In [5]:
# Aggregate to one row per inspection
# Count total violations and critical violations per inspection

inspection_agg = df_clean.groupby(['CAMIS', 'INSPECTION DATE']).agg({
    'DBA': 'first',                           # Restaurant name
    'BORO': 'first',                          # Borough  
    'ZIPCODE': 'first',                       # Zip code
    'CUISINE DESCRIPTION': 'first',           # Cuisine type
    'SCORE': 'first',                         # Score (same for all violations)
    'GRADE': 'first',                         # Grade (same for all violations)
    'VIOLATION CODE': 'count',                # Total violations
    'CRITICAL FLAG': lambda x: (x == 'Critical').sum()  # Critical violations
}).reset_index()

# Rename columns
inspection_agg.columns = [
    'CAMIS', 'INSPECTION_DATE', 'RESTAURANT_NAME', 'BORO', 
    'ZIPCODE', 'CUISINE', 'SCORE', 'GRADE', 
    'TOTAL_VIOLATIONS', 'CRITICAL_VIOLATIONS'
]

print(f"Aggregated shape: {inspection_agg.shape}")
inspection_agg.head()

Aggregated shape: (51854, 10)


,CAMIS,INSPECTION_DATE,RESTAURANT_NAME,BORO,ZIPCODE,CUISINE,SCORE,GRADE,TOTAL_VIOLATIONS,CRITICAL_VIOLATIONS
0,30075445,2023-08-22,MORRIS PARK BAKE SHOP,Bronx,10462.0,Bakery Products/Desserts,12.0,A,3,1
1,30075445,2024-11-08,MORRIS PARK BAKE SHOP,Bronx,10462.0,Bakery Products/Desserts,10.0,A,3,1
2,30191841,2023-04-23,D.J. REYNOLDS,Manhattan,10019.0,Irish,10.0,A,2,2
3,30191841,2025-02-20,D.J. REYNOLDS,Manhattan,10019.0,Irish,10.0,A,2,2
4,40356018,2024-04-16,RIVIERA CATERERS,Brooklyn,11224.0,American,0.0,A,0,0


In [6]:
# Add time features
inspection_agg['YEAR'] = inspection_agg['INSPECTION_DATE'].dt.year
inspection_agg['MONTH'] = inspection_agg['INSPECTION_DATE'].dt.month
inspection_agg['DAY_OF_WEEK'] = inspection_agg['INSPECTION_DATE'].dt.dayofweek  # 0=Monday
inspection_agg['QUARTER'] = inspection_agg['INSPECTION_DATE'].dt.quarter

inspection_agg.head()

,CAMIS,INSPECTION_DATE,RESTAURANT_NAME,BORO,ZIPCODE,CUISINE,SCORE,GRADE,TOTAL_VIOLATIONS,CRITICAL_VIOLATIONS,YEAR,MONTH,DAY_OF_WEEK,QUARTER
0,30075445,2023-08-22,MORRIS PARK BAKE SHOP,Bronx,10462.0,Bakery Products/Desserts,12.0,A,3,1,2023,8,1,3
1,30075445,2024-11-08,MORRIS PARK BAKE SHOP,Bronx,10462.0,Bakery Products/Desserts,10.0,A,3,1,2024,11,4,4
2,30191841,2023-04-23,D.J. REYNOLDS,Manhattan,10019.0,Irish,10.0,A,2,2,2023,4,6,2
3,30191841,2025-02-20,D.J. REYNOLDS,Manhattan,10019.0,Irish,10.0,A,2,2,2025,2,3,1
4,40356018,2024-04-16,RIVIERA CATERERS,Brooklyn,11224.0,American,0.0,A,0,0,2024,4,1,2


In [7]:
# Clean borough names
boro_map = {
    'Manhattan': 'Manhattan',
    'Bronx': 'Bronx', 
    'Brooklyn': 'Brooklyn',
    'Queens': 'Queens',
    'Staten Island': 'Staten Island',
    '1': 'Manhattan',
    '2': 'Bronx',
    '3': 'Brooklyn',
    '4': 'Queens',
    '5': 'Staten Island'
}

inspection_agg['BORO'] = inspection_agg['BORO'].astype(str).map(boro_map)
inspection_agg['BORO'].value_counts()

BORO
Manhattan        19881
Brooklyn         13521
Queens           11789
Bronx             4769
Staten Island     1879
Name: count, dtype: int64

In [8]:
# Check missing values
print(inspection_agg.isnull().sum())

CAMIS                    0
INSPECTION_DATE          0
RESTAURANT_NAME          0
BORO                    15
ZIPCODE                679
CUISINE                  0
SCORE                    0
GRADE                    0
TOTAL_VIOLATIONS         0
CRITICAL_VIOLATIONS      0
YEAR                     0
MONTH                    0
DAY_OF_WEEK              0
QUARTER                  0
dtype: int64


In [9]:
# Drop rows with missing essential columns
inspection_agg = inspection_agg.dropna(subset=['SCORE', 'GRADE', 'CUISINE', 'BORO'])
print(f"Final shape: {inspection_agg.shape}")

Final shape: (51839, 14)


In [10]:
# Final check
print(f"\nGrade distribution:")
print(inspection_agg['GRADE'].value_counts())

print(f"\nScore stats:")
print(inspection_agg['SCORE'].describe())


Grade distribution:
GRADE
A    45013
B     4475
C     2351
Name: count, dtype: int64

Score stats:
count    51839.000000
mean        11.636471
std          8.055486
min          0.000000
25%          8.000000
50%         11.000000
75%         13.000000
max        154.000000
Name: SCORE, dtype: float64


In [11]:
# Save clean data
inspection_agg.to_csv('../data/inspections_clean.csv', index=False)
print("Saved to ../data/inspections_clean.csv")

Saved to ../data/inspections_clean.csv
